In [32]:
import re
from copy  import deepcopy

In [249]:
def read(file:str):
    data = {}
    with open(file) as f:
        for line in f:
            p = line.strip().split()
            data[p[1]] = [int(p[4].split("=")[-1].strip(";")), [x.strip(",") for x in p[9:]]]
    return data

inputs = read("inputs/day-16-example.txt")

In [260]:
inputs

{'AA': [0, ['DD', 'II', 'BB']],
 'BB': [13, ['CC', 'AA']],
 'CC': [2, ['DD', 'BB']],
 'DD': [20, ['CC', 'AA', 'EE']],
 'EE': [3, ['FF', 'DD']],
 'FF': [0, ['EE', 'GG']],
 'GG': [0, ['FF', 'HH']],
 'HH': [22, ['GG']],
 'II': [0, ['AA', 'JJ']],
 'JJ': [21, ['II']]}

## PArt 1

In [279]:
t = 30
beams = [[0,[], ["AA"], inputs]]
while t > 0:
    new_beams = []
    # Take all actions for each beam, keep the top N
    for value, open, steps, data in beams:
        cur = steps[0]
        # Open
        if data[cur][0] > 0:
            nd = deepcopy(data)
            nd[cur][0] = 0
            new_beams += [[value + (t - 1) * data[cur][0], [(cur,30-t+1)]+open, [cur] + steps, nd]]
        # Neighbors
        for nn in data[cur][-1]:
            new_beams += [[value, open, [nn] + steps, data]]
    v0 = max(b[0] for b in beams)
    beams = sorted(new_beams, key=lambda x: x[0], reverse=True)[:500]
    v1 = max(b[0] for b in beams)
    # print(v1, v0, v1 - v0)
    t -= 1


In [280]:
beams[0][:2]

[1651, [('CC', 24), ('EE', 21), ('HH', 17), ('JJ', 9), ('BB', 5), ('DD', 2)]]

In [92]:
beams

[[0, 'AA', [0, ['DD', 'II', 'BB']]]]

## Part 2

In [ ]:
t = 26
beams = [[0, [[], []], [["AA"], ["AA"]], inputs]]
while t > 0:
    new_beams = []
    # Take all actions for each beam, keep the top N
    for value, (oa, ob), (sa, sb), data in beams:
        ca, cb = sa[0], sb[0]

        # Both open

        # Both move
        for na in data[ca][-1]:
            for nb in data[ca][-1]:
                new_beams += [[value, open, [nn] + steps, data]]



    #     cur = steps[0]
    #     # Open
    #     if data[cur][0] > 0:
    #         nd = deepcopy(data)
    #         nd[cur][0] = 0
    #         new_beams += [
    #             [
    #                 value + (t - 1) * data[cur][0],
    #                 [(cur, 30 - t + 1)] + open,
    #                 [cur] + steps,
    #                 nd,
    #             ]
    #         ]
    #     # Neighbors
    #     for nn in data[cur][-1]:
    #         new_beams += [[value, open, [nn] + steps, data]]
    # v0 = max(b[0] for b in beams)
    # beams = sorted(new_beams, key=lambda x: x[0], reverse=True)[:500]
    # v1 = max(b[0] for b in beams)
    # # print(v1, v0, v1 - v0)
    t -= 1


In [251]:
t = 26
beams = [[0, [["AA"], ["AA"]], inputs]]
while t > 0:
    new_beams = []
    # Take all actions for each beam, keep the top N
    for value, [sa, sb], data in beams:
        if all(d[0] == 0 for d in data.values()):
            new_beams += [[value, [sa, sb], data]]
            continue
        ca, cb = sa[0], sb[0]

        # Both open
        na = deepcopy(data)
        v = (t - 1) * na[ca][0]
        na[ca][0] = 0
        v += (t - 1) * na[cb][0]
        na[cb][0] = 0
        new_beams += [[v, [[ca] + sa, [cb] + sb], na]]

        # open, move
        na = deepcopy(data)
        na[ca][0] = 0
        for nn in data[cb][-1]:
            new_beams += [[value + (t - 1) * data[ca][0], [[ca] + sa, [nn] + sb], na]]
        nb = deepcopy(data)
        nb[cb][0] = 0
        for nn in data[ca][-1]:
            new_beams += [[value + (t - 1) * data[cb][0], [[nn] + sa, [cb] + sb], nb]]

        # move, move
        for na in data[ca][-1]:
            for nb in data[cb][-1]:
                new_beams += [[value, [[na] + sa, [nb] + sb], data]]
    beams = sorted(new_beams, key=lambda x: x[0], reverse=True)[:1000]
    # if t >= 25:
    #     print([(b[0], b[1]) for b in beams[:10]])
    t -= 1


In [252]:
# 1000: 1890, 2048: 2085, 4096: 2085, 10_000: 2085
beams[0][0]

1706

In [54]:
bw = 5
paths = {}
t = 30
cur = "A"
while t > 0:
    


SyntaxError: incomplete input (1401637043.py, line 6)

In [34]:
data = deepcopy(inputs)
cur = "AA"
t = 30
reward = 0
while t > 0:
    # Greedy
    open_reward = data[cur][0]*(t-1)
    move_rewards = {n: data[n][0]*(t-2) for n in data[cur][-1]}
    print("check", open_reward, move_rewards)
    if open_reward > max(move_rewards.values()):
        print("open", cur, open_reward)
        reward += open_reward
        data[cur][0] = 0
    else:
        cur = max(move_rewards, key=lambda k: move_rewards[k])
        print("move", cur)
    t -= 1


check 0 {'DD': 560, 'II': 0, 'BB': 364}
move DD
check 560 {'CC': 54, 'AA': 0, 'EE': 81}
open DD 560
check 0 {'CC': 52, 'AA': 0, 'EE': 78}
move EE
check 78 {'FF': 0, 'DD': 0}
open EE 78
check 0 {'FF': 0, 'DD': 0}
move FF
check 0 {'EE': 0, 'GG': 0}
move EE
check 0 {'FF': 0, 'DD': 0}
move FF
check 0 {'EE': 0, 'GG': 0}
move EE
check 0 {'FF': 0, 'DD': 0}
move FF
check 0 {'EE': 0, 'GG': 0}
move EE
check 0 {'FF': 0, 'DD': 0}
move FF
check 0 {'EE': 0, 'GG': 0}
move EE
check 0 {'FF': 0, 'DD': 0}
move FF
check 0 {'EE': 0, 'GG': 0}
move EE
check 0 {'FF': 0, 'DD': 0}
move FF
check 0 {'EE': 0, 'GG': 0}
move EE
check 0 {'FF': 0, 'DD': 0}
move FF
check 0 {'EE': 0, 'GG': 0}
move EE
check 0 {'FF': 0, 'DD': 0}
move FF
check 0 {'EE': 0, 'GG': 0}
move EE
check 0 {'FF': 0, 'DD': 0}
move FF
check 0 {'EE': 0, 'GG': 0}
move EE
check 0 {'FF': 0, 'DD': 0}
move FF
check 0 {'EE': 0, 'GG': 0}
move EE
check 0 {'FF': 0, 'DD': 0}
move FF
check 0 {'EE': 0, 'GG': 0}
move EE
check 0 {'FF': 0, 'DD': 0}
move FF
check 0 {'

In [35]:
reward

638